<랜덤 포레스트 알고리즘을 활용한 스마트시티 표준 분류 모델>
저번에 사용한 랜덤포레스트와 비교하여 좀 더 오버피팅을 방지하였음

1. **데이터 전처리 및 준비**
* 데이터 로딩: ETRI 표준 분류에서 특정 칼럼('적용단계', '스마트 인프라', '인프라분류', '상호운용성수준', '세부분류키워드')을 선택하여 JSON 파일로 저장한다.
* 결측값 처리: 행 중에서 결측값이 있는 것을 제거한다. 현재 주어진 데이터에서 결측값은 null값을 의미한다. 총 4700여개의 데이터 중 null값이 없는 1240개의 데이터만 사용하였음.
* 레이블 인코딩: 카테고리형 변수(문자형 변수)를 정수로 변환한다.
* 데이터 분할: 전처리된 데이터를 훈련 세트(80%)와 테스트 세트(20%)로 분할한다.

---


2. **하이퍼파라미터 튜닝**
* 그리드 서치: 'n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf' 등의 하이퍼파라미터에 대한 여러 가지 조합을 시도하여 최적의 하이퍼파라미터를 찾는다.


---



3. **모델 훈련 및 평가**
* 모델 초기화: 랜덤 포레스트 분류기를 초기화한다.
* 모델 훈련: 훈련 세트를 사용하여 랜덤 포레스트 분류기를 훈련시킨다.
* 모델 평가: 테스트 세트를 사용하여 모델의 성능을 평가한다. 주로 정확도를 사용한다.


---


4. **교차 검증 및 분류 모델 최종성능 확인**
5-겹 교차 검증: 최적의 하이퍼파라미터를 사용한 모델에 대해 5-겹 교차 검증을 수행하여 모델의 일반화 성능을 평가한다.
=> 쉽게 설명하면 데이터를 5개의 부분집합으로 나누고 훈련 셋(4개의 부분집합)과 테스트 셋(1개의 부분집합)을 변경해가며 사용하여 검증하는 방법.

---

5. **모델 저장, 모델 사용 예시 출력**
Joblib 사용: 훈련된 모델을 파일로 저장하기 위해 Joblib 라이브러리를 사용한다.


---




1. 데이터 전처리 및 준비

In [12]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import joblib

# delete 'space' from data
def normalize_string(s):
    return s.lower().replace(" ", "")


# Load and clean the data
df_cleaned = pd.read_json('/content/drive/MyDrive/etri_drive/data/xyz_selected_data_extracted.json').dropna()
df_cleaned['적용단계'] = df_cleaned['적용단계'].str.strip()
label_encoders = {}
for column in df_cleaned.select_dtypes(include=['object']).columns:
    df_cleaned[column] = df_cleaned[column].apply(normalize_string)
    le = LabelEncoder()
    df_cleaned[column] = le.fit_transform(df_cleaned[column])
    label_encoders[column] = le

# Split data into training and test sets
X = df_cleaned.drop('적용단계', axis=1)
y = df_cleaned['적용단계']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest classifier
rf_clf = RandomForestClassifier(random_state=42)

print('step1 complete')

step1 complete


2.하이퍼 파라미터 튜닝

In [13]:
# Hyperparameter Tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

print('step2 complete')

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


step2 complete


3. 모델 훈련 및 평가





In [14]:

# Train and evaluate Random Forest with best parameters
best_rf_clf = RandomForestClassifier(**best_params, random_state=42)
best_rf_clf.fit(X_train, y_train)
y_pred = best_rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(accuracy);
print('step3 complete')


0.6465927099841522
step3 complete


4. 훈련 셋과 검증 셋을 합친 후, 검증 셋에 나타나는 새로운 조합을 고려한 모델을 훈련시킨다.

In [15]:


# Combine the training and test feature sets and labels
all_features = pd.concat([X_train, X_test])
all_labels = pd.concat([y_train, y_test])

# Combine them into a single DataFrame
all_data = pd.concat([all_features, all_labels], axis=1)

# Remove duplicate rows based on the feature columns to get unique combinations
unique_combinations_all = all_data.drop('적용단계', axis=1).drop_duplicates()

# Prepare the corresponding labels for these unique combinations
y_unique_combinations_all = all_data.loc[unique_combinations_all.index, '적용단계']

# Train the Random Forest classifier with best parameters on all unique combinations
best_rf_clf.fit(unique_combinations_all, y_unique_combinations_all)

# Predict probabilities for these unique combinations
predicted_probs_unique_combinations_all = best_rf_clf.predict_proba(unique_combinations_all)

# Create a DataFrame to hold the predicted probabilities and convert them to percentage
prob_df_unique_combinations_all = pd.DataFrame(predicted_probs_unique_combinations_all,
                                               columns=label_encoders['적용단계'].inverse_transform(best_rf_clf.classes_)) * 100

# Decode the feature values for easier interpretation
decoded_unique_combinations_all = unique_combinations_all.copy()
for column in decoded_unique_combinations_all.columns:
    decoded_unique_combinations_all[column] = label_encoders[column].inverse_transform(decoded_unique_combinations_all[column])

# Combine decoded feature values and predicted probabilities
combined_result_unique_all = pd.concat([decoded_unique_combinations_all.reset_index(drop=True), prob_df_unique_combinations_all.reset_index(drop=True)], axis=1)

# Save the DataFrame to an Excel file
excel_file_path_all = '/content/drive/MyDrive/etri_drive/result/predicted_probabilities_all.xlsx'
combined_result_unique_all.to_excel(excel_file_path_all, index=False)



---

아래는 기존에 classifier를 만들 때 사용하였음


---



In [ ]:


# Perform 5-Fold Cross-Validation
cv_scores = cross_val_score(best_rf_clf, X, y, cv=5)
cv_mean = cv_scores.mean()
cv_std = cv_scores.std()

# Print the accuracy on the test set
print(f"Test set accuracy: {accuracy * 100:.2f}%")

# Print the 5-Fold Cross-Validation scores
print(f"5-Fold CV Scores: {cv_scores}")

# Print the mean and standard deviation of the CV scores
print(f"5-Fold CV Mean: {cv_mean * 100:.2f}%")
print(f"5-Fold CV Standard Deviation: {cv_std * 100:.2f}%")

print('step4 complete')


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Test set accuracy: 70.56%
5-Fold CV Scores: [0.47983871 0.63306452 0.66935484 0.70564516 0.84615385]
5-Fold CV Mean: 66.68%
5-Fold CV Standard Deviation: 11.81%
step4 complete


In [ ]:

# Save the trained model
model_file_path = '/content/drive/MyDrive/etri_drive/models/model.joblib'
joblib.dump(best_rf_clf, model_file_path)

print('save complete')

save complete


In [ ]:

# Load the trained model from the file
loaded_model = joblib.load(model_file_path)
print('model load complete')
# Select a few samples from the test set
sample_data = X_test.sample(5, random_state=42)

# Make predictions using the loaded model
sample_predictions = loaded_model.predict(sample_data)

# Decode the sample data categories back to original text labels
decoded_sample_data = sample_data.copy()
for column in sample_data.columns:
    decoded_sample_data[column] = label_encoders[column].inverse_transform(sample_data[column])

# Show the decoded sample data along with the decoded predictions
decoded_sample_data, decoded_predictions






model load complete


NameError: ignored